# Mount Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries and initializing stopwords and stemmer


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

import nltk

import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
nltk.download('wordnet')




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Reading Data


In [ ]:
hasoc_data = pd.read_csv("drive/MyDrive/ICHCL_baseline/hasoc2021-train/Hasoc2021_train.csv") #Your respective address
hasoc_data.head()


,Unnamed: 0,_id,text,task_1,task_2
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN


In [ ]:
aakash_test = pd.read_csv("drive/MyDrive/ICHCL_baseline/Test/aakash_300.csv")

In [ ]:
aakash_test.describe()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
aakash_test.head()

,_id,text,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,task_1,task_2
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations. So sa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT,NONE
1,60c5d6bf5659ea5e55df028c,@MattHancock This may all be true. But... What...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOF,PRFN
2,60c5d6bf5659ea5e55def377,@Layla_EFC I’ve unfollowed him the wanker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOF,OFFN
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world. The...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOF,OFFN
4,60c5d6bf5659ea5e55df01a6,"And thus death laughs... It is sad merriment, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT,NONE


In [ ]:
test_hasoc = pd.read_csv("drive/MyDrive/ICHCL_baseline/Test/en_Hasoc2021_test_task1.csv")
test_hasoc.head()

,_id,text
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations. So sa...
1,60c5d6bf5659ea5e55df028c,@MattHancock This may all be true. But... What...
2,60c5d6bf5659ea5e55def377,@Layla_EFC I’ve unfollowed him the wanker
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world. The...
4,60c5d6bf5659ea5e55df01a6,"And thus death laughs... It is sad merriment, ..."


In [ ]:
hasoc_data.describe()

,Unnamed: 0
count,3843.000000
mean,2558.799375
std,1477.088070
min,1.000000
25%,1276.500000
50%,2574.000000
75%,3828.000000
max,5121.000000


In [ ]:
tweets = hasoc_data.text
y_task1 = hasoc_data.task_1
y_task2 = hasoc_data.task_2

# Basic Preprocessing of Tweets


In [ ]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
def preprocess_text(text):
    text = re.sub(r"http\S+", "", text)             # removing website links
    text = BeautifulSoup(text, 'lxml').get_text()   # removing html tags
    text = decontract(text)                         # decontracting
    text = re.sub("\S*\d\S*", "", text).strip()     # removing the words with numeric digits
    text = re.sub('[^A-Za-z]+', ' ', text)          # removing non-word characters
    text = text.lower()                             # converting to lower case
    text = [word for word in text.split(" ") if not word in stop_words] # removing stop words
    text = [lemmatizer.lemmatize(token, "v") for token in text] #Lemmatization
    text = " ".join(text)
    text.strip()
    return text

def convert_task(text):
    if text == "HOF":
      return 1
    return 0

In [ ]:
hasoc_data["text"] = hasoc_data["text"].apply(lambda x: preprocess_text(x))
hasoc_data["task_1"] = hasoc_data["task_1"].apply(lambda x: convert_task(x))
aakash_test["text"] = aakash_test["text"].apply(lambda x: preprocess_text(x))
aakash_test["task_1"] = aakash_test["task_1"].apply(lambda x: convert_task(x))
test_hasoc["text"] = test_hasoc["text"].apply(lambda x: preprocess_text(x))


In [ ]:
hasoc_data["text"].head()

0     wealth make not able start make money sustain...
1          technically still turn back clock dick head
2     vmbjp narendramodi jpnadda amitshah dilipghos...
3               krtoprak yigit soldier japan dick head
4     blueheartedly would better ask think sleazy s...
Name: text, dtype: object

In [ ]:
test_hasoc["text"].head()

0    fewer people come vaccinations sad nurse covid...
1                   matthancock may true piss big dom 
2                          layla efc unfollowed wanker
3               guy lose world jung label islamophobia
4    thus death laugh sad merriment still covid ind...
Name: text, dtype: object

In [ ]:
hasoc_data["task_1"].head()

0    1
1    1
2    0
3    1
4    1
Name: task_1, dtype: int64

In [ ]:
hasoc_data["task_1"].value_counts()

1    2501
0    1342
Name: task_1, dtype: int64

In [ ]:
hasoc_data.drop(["Unnamed: 0","_id","task_2"],inplace=True,axis=1)

In [ ]:
hasoc_data.head()

,text,task_1
0,wealth make not able start make money sustain...,1
1,technically still turn back clock dick head,1
2,vmbjp narendramodi jpnadda amitshah dilipghos...,0
3,krtoprak yigit soldier japan dick head,1
4,blueheartedly would better ask think sleazy s...,1


In [ ]:
train_df = hasoc_data
test_df = aakash_test

print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)
print("Test size of hasoc submission",test_hasoc.shape)

Training data size :  (3843, 2)
Test data size :  (300, 12)
Test size of hasoc submission (1281, 2)


In [ ]:
train_df["task_1"].value_counts()


1    2501
0    1342
Name: task_1, dtype: int64

##Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True, stop_words='english')
X = cv.fit_transform(train_df.text)
y_train = train_df.task_1
X_test = cv.transform(test_df.text)
X_test_submission = cv.transform(test_hasoc.text)
y_test = test_df.task_1

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
X = tfidf.fit_transform(X)
print(X.shape)
X_test = tfidf.transform(X_test)
X_test_submission = tfidf.transform(X_test_submission)

(3843, 10487)


#Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=2000)
# rf.fit(X, y_train)
# y_pred = rf.predict(X_test)
# y_submission = rf.predict(X_test_submission)
# print ("Accuracy: %s" 
#        % accuracy_score(y_test,y_pred))

In [ ]:
# print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
# print('F1-score: ', f1_score(y_pred, y_test))
# print('Confusion matrix:')
# confusion_matrix(y_test,y_pred)

In [ ]:
# print(classification_report(y_test, y_pred))

# Logistic Regression Model

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y_task1, test_size=0.3, random_state=42)
X_train = X
X_val = X_test
y_train = hasoc_data["task_1"]


In [ ]:
classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

In [ ]:
# y_pred = classifier.predict(X_val)

In [ ]:
# print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
# print('F1-score: ', f1_score(y_pred, y_test))
# print('Confusion matrix:')
# confusion_matrix(y_test,y_pred)

In [ ]:
# print(classification_report(y_test, y_pred))

#SVM model


In [ ]:
SVM = SVC(kernel='rbf',probability=True)
SVM.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
svm_pred = SVM.predict(X_test)

In [ ]:
print("Accuracy of the model : ", accuracy_score(svm_pred, y_test))
print('F1-score: ', f1_score(svm_pred, y_test))
print('Confusion matrix:')
confusion_matrix(svm_pred,y_test)

Accuracy of the model :  0.81
F1-score:  0.87527352297593
Confusion matrix:


array([[ 43,  30],
       [ 27, 200]])

In [ ]:
print(classification_report(y_test, svm_pred))

              precision    recall  f1-score   support

           0       0.59      0.61      0.60        70
           1       0.88      0.87      0.88       230

    accuracy                           0.81       300
   macro avg       0.74      0.74      0.74       300
weighted avg       0.81      0.81      0.81       300



#Ensemble Model

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
clf3 = GaussianNB()

Ensemble 1

In [ ]:
eclf1 = VotingClassifier(estimators=[
         ('lr', classifier), ('rf', rf), ('svm',SVM)], voting='hard',weights=[1,1,3],
        flatten_transform=True)
eclf1.fit(X, y_train)

ens_pred1 = eclf1.predict(X_test)

In [ ]:
print("Accuracy of the model : ", accuracy_score(ens_pred1, y_test))
print('F1-score: ', f1_score(ens_pred1, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,ens_pred1)

Accuracy of the model :  0.81
F1-score:  0.87527352297593
Confusion matrix:


array([[ 43,  27],
       [ 30, 200]])

In [ ]:
print(classification_report(y_test, ens_pred1))

              precision    recall  f1-score   support

           0       0.59      0.61      0.60        70
           1       0.88      0.87      0.88       230

    accuracy                           0.81       300
   macro avg       0.74      0.74      0.74       300
weighted avg       0.81      0.81      0.81       300



Ensemble 2

In [ ]:
eclf2 = VotingClassifier(estimators=[
         ('lr', classifier), ('rf', rf), ('svm',SVM)],
         voting='soft')

eclf2.fit(X, y_train)

ens_pred2 = eclf2.predict(X_test)

In [ ]:
print("Accuracy of the model : ", accuracy_score(ens_pred2, y_test))
print('F1-score: ', f1_score(ens_pred2, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,ens_pred2)

Accuracy of the model :  0.7933333333333333
F1-score:  0.8603603603603603
Confusion matrix:


array([[ 47,  23],
       [ 39, 191]])

In [ ]:
print(classification_report(y_test, ens_pred2))

              precision    recall  f1-score   support

           0       0.55      0.67      0.60        70
           1       0.89      0.83      0.86       230

    accuracy                           0.79       300
   macro avg       0.72      0.75      0.73       300
weighted avg       0.81      0.79      0.80       300



Ensemble 3

In [ ]:
eclf3 = VotingClassifier(estimators=[
       ('lr', classifier), ('rf', rf), ('svm',SVM)],
        voting='soft', weights=[2,1,1],
        flatten_transform=True)
eclf3.fit(X, y_train)

ens_pred3 = eclf3.predict(X_test)

In [ ]:
print("Accuracy of the model : ", accuracy_score(ens_pred3, y_test))
print('F1-score: ', f1_score(ens_pred3, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,ens_pred3)

Accuracy of the model :  0.79
F1-score:  0.8584269662921348
Confusion matrix:


array([[ 46,  24],
       [ 39, 191]])

In [ ]:
print(classification_report(y_test, ens_pred3))

              precision    recall  f1-score   support

           0       0.54      0.66      0.59        70
           1       0.89      0.83      0.86       230

    accuracy                           0.79       300
   macro avg       0.71      0.74      0.73       300
weighted avg       0.81      0.79      0.80       300



Ensemble 4

In [ ]:
eclf4 = VotingClassifier(estimators=[
       ('lr', classifier), ('rf', rf),('svm',SVM)],
        voting='soft', weights=[1,2,1],
        flatten_transform=True)
eclf4.fit(X, y_train)

ens_pred4 = eclf4.predict(X_test)

In [ ]:
print("Accuracy of the model : ", accuracy_score(ens_pred4, y_test))
print('F1-score: ', f1_score(ens_pred4, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,ens_pred4)

Accuracy of the model :  0.7966666666666666
F1-score:  0.8629213483146067
Confusion matrix:


array([[ 47,  23],
       [ 38, 192]])

In [ ]:
print(classification_report(y_test, ens_pred4))

              precision    recall  f1-score   support

           0       0.55      0.67      0.61        70
           1       0.89      0.83      0.86       230

    accuracy                           0.80       300
   macro avg       0.72      0.75      0.73       300
weighted avg       0.81      0.80      0.80       300



Ensemble 5

In [ ]:
eclf5 = VotingClassifier(estimators=[
       ('lr', classifier), ('rf', rf),('svm',SVM)],
        voting='soft', weights=[1,1,2],
        flatten_transform=True)
eclf5.fit(X, y_train)

ens_pred5 = eclf5.predict(X_test)

In [ ]:
print("Accuracy of the model : ", accuracy_score(ens_pred5, y_test))
print('F1-score: ', f1_score(ens_pred5, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,ens_pred5)

Accuracy of the model :  0.79
F1-score:  0.8577878103837472
Confusion matrix:


array([[ 47,  23],
       [ 40, 190]])

In [ ]:
print(classification_report(y_test, ens_pred5))

              precision    recall  f1-score   support

           0       0.54      0.67      0.60        70
           1       0.89      0.83      0.86       230

    accuracy                           0.79       300
   macro avg       0.72      0.75      0.73       300
weighted avg       0.81      0.79      0.80       300

